In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

In [2]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Convert grayscale images to RGB (3 channels)
x_train = tf.image.grayscale_to_rgb(tf.expand_dims(x_train, axis=-1))
x_test = tf.image.grayscale_to_rgb(tf.expand_dims(x_test, axis=-1))

# Resize images to a smaller size like 128x128 instead of 224x224
x_train = tf.image.resize(x_train, [128, 128])
x_test = tf.image.resize(x_test, [128, 128])

# Normalize pixel values to the range of 0 to 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [ ]:
# Load the VGG16 model with pretrained ImageNet weights
# Exclude the top (classification) layers to customize for Fashion MNIST
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
# Add custom classification layers for Fashion MNIST
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

In [7]:
# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 256
epochs = 7

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=batch_size,
    epochs=epochs
)

Epoch 1/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1661s 7s/step - accuracy: 0.8045 - loss: 0.8179 - val_accuracy: 0.8857 - val_loss: 0.3208
Epoch 2/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1680s 7s/step - accuracy: 0.9030 - loss: 0.2663 - val_accuracy: 0.9011 - val_loss: 0.2729
Epoch 3/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1640s 7s/step - accuracy: 0.9225 - loss: 0.2149 - val_accuracy: 0.9057 - val_loss: 0.2601
Epoch 4/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1805s 8s/step - accuracy: 0.9336 - loss: 0.1876 - val_accuracy: 0.9034 - val_loss: 0.2671
Epoch 5/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1687s 7s/step - accuracy: 0.9384 - loss: 0.1704 - val_accuracy: 0.9052 - val_loss: 0.2653
Epoch 6/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1594s 7s/step - accuracy: 0.9405 - loss: 0.1618 - val_accuracy: 0.8995 - val_loss: 0.2725
Epoch 7/7
235/235 ━━━━━━━━━━━━━━━━━━━━ 1626s 7s/step - accuracy: 0.9457 - loss: 0.1486 - val_accuracy: 0.8984 - val_loss: 0.2977


In [8]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 235s 750ms/step - accuracy: 0.8976 - loss: 0.2902
Test Accuracy: 89.84%
